In [1]:
# Importação das bibliotecas 
import numpy as np # Essa biblioteca consiste em objetos chamados de arrays (matrizes), que são multidimensionais
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import os
import glob # É usado para retornar todos os caminhos de arquivo que correspondem a um padrão específico.
import plotly.graph_objects as go
import cv2
import warnings
from PIL import Image
from PIL import ImageFile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from tensorflow.keras.optimizers import Adam
from keras.optimizers import adam_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten , Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
warnings.filterwarnings("ignore")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [25]:
# Carregando Dataset 

root_dir = 'D:\GIT\ProjetoII\dataset' # Acessando diretorio de onde está localizado o dataset que iremos trabalhar.
treino_dir = os.path.join(root_dir,'treino') # Método para unir um ou mais componentes de caminho de forma inteligente.
print('''Caminho do dadaset é : ''',treino_dir)
print()

tipo1_dir = os.path.join(treino_dir, 'tipo_1')# Acessando diretorio "Tipo_*", onde estão nosssa imagens.
tipo2_dir = os.path.join(treino_dir, 'tipo_2')
tipo3_dir = os.path.join(treino_dir, 'tipo_3')
print('''Caminho do dadaset para o tipo 1 é : ''',tipo1_dir)
print()

treino_tipo1_files = glob.glob(tipo1_dir+'/*.jpg') # Encontra todos os nomes de caminho que correspondem a um padrão especificado de acordo com as regras usadas pelo shell Unix
treino_tipo2_files = glob.glob(tipo2_dir+'/*.jpg')
treino_tipo3_files = glob.glob(tipo3_dir+'/*.jpg')


added_tipo1_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_1", "tipo_1")+'/*.jpg')
added_tipo2_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_2", "tipo_2")+'/*.jpg')
added_tipo3_files  =  glob.glob(os.path.join(root_dir, "adicionar_tipo_3", "tipo_3")+'/*.jpg')


tipo1_files = treino_tipo1_files + added_tipo1_files
tipo2_files = treino_tipo2_files + added_tipo2_files
tipo3_files = treino_tipo3_files + added_tipo3_files

print(f'''Foram adicionado {len(tipo1_files)} imagens do Tipo 1  
Foram adicionado {len(tipo2_files)} imagens do Tipo 2
Foram adicionado {len(tipo3_files)} imagens do Tipo 3''' )

Caminho do dadaset é :  D:\GIT\ProjetoII\dataset\treino

Caminho do dadaset para o tipo 1 é :  D:\GIT\ProjetoII\dataset\treino\tipo_1

Foram adicionado 1441 imagens do Tipo 1  
Foram adicionado 4348 imagens do Tipo 2
Foram adicionado 2426 imagens do Tipo 3


In [35]:
# Criar dataframe dos arquivo e rótulos
arquivos = {'caminho': tipo1_files + tipo2_files + tipo3_files,
          'rotulos': ['Tipo 1']* len(tipo1_files) + ['Tipo 2']* len(tipo2_files) + ['Tipo 3']* len(tipo3_files)}

cance_df = pd.DataFrame(arquivos).sample(frac=1, random_state= 1).reset_index(drop=True)
cance_df

,caminho,rotulos
0,D:\GIT\ProjetoII\dataset\treino\tipo_1\1414.jpg,Tipo 1
1,D:\GIT\ProjetoII\dataset\adicionar_tipo_3\tipo...,Tipo 3
2,D:\GIT\ProjetoII\dataset\adicionar_tipo_1\tipo...,Tipo 1
3,D:\GIT\ProjetoII\dataset\adicionar_tipo_2\tipo...,Tipo 2
4,D:\GIT\ProjetoII\dataset\adicionar_tipo_3\tipo...,Tipo 3
...,...,...
8210,D:\GIT\ProjetoII\dataset\adicionar_tipo_2\tipo...,Tipo 2
8211,D:\GIT\ProjetoII\dataset\adicionar_tipo_3\tipo...,Tipo 3
8212,D:\GIT\ProjetoII\dataset\adicionar_tipo_1\tipo...,Tipo 1
8213,D:\GIT\ProjetoII\dataset\adicionar_tipo_2\tipo...,Tipo 2


In [36]:
# Describe(), método calcula e exibe estatísticas resumidas para um dataframe
cance_df.describe()

,caminho,rotulos
count,8215,8215
unique,8215,3
top,D:\GIT\ProjetoII\dataset\treino\tipo_1\1070.jpg,Tipo 2
freq,1,4348


In [37]:
# check for duplicates
len(cance_df[cance_df.duplicated(subset=['caminho'])])

0

In [40]:
# Verificando se existe arquivos danificado
bad_files = []
for path in (cance_df['caminho'].values):
    try:
        img = Image.open(path)
    except:
        index = cance_df[cance_df['caminho']==path].index.values[0]
        bad_files.append(index)
print(len(bad_files))

3


In [41]:
# Deletando arquivos danificados
cance_df.drop(bad_files, inplace=True)

In [42]:
# check length of files in dataframe
##len(cance_df)

8212

In [43]:
# Verficando tipos de rotulos
cance_df['rotulos'].unique()

array(['Tipo 1', 'Tipo 3', 'Tipo 2'], dtype=object)

In [45]:
# Fazendo um contagem de cada tipo de rotulo
tipo_count = pd.DataFrame(cance_df['rotulos'].value_counts()).rename(columns= {'rotulos': ' Quantidades'})
tipo_count

,Quantidades
Tipo 2,4346
Tipo 3,2426
Tipo 1,1440
